<a href="https://colab.research.google.com/github/KarinaRmzG/Neuronal-Networks/blob/main/Ejercicio7_Backpropagation_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Entrenamiento con retropropagación
En este ejercicio implementaremos el algoritmo de retropropagación dentro del descenso por gradiente para actualizar todos los pesos de la red durante varias épocas. Para entrenar la red usaremos el conjunto de datos de calificaciones que vimos previamente.

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Contenido del archivo "data_prep.py"

In [2]:
import numpy as np
import pandas as pd

# Leer un archivo CSV llamado 'data.csv' ubicado en la ruta '/content/drive/MyDrive/Ejercicios IA/'
# y cargarlo en un DataFrame llamado 'admissions'
admissions = pd.read_csv('/content/drive/MyDrive/Ejercicios IA/data.csv')

# Crear variables ficticias para la columna 'rank' y agregarlas al DataFrame 'data'
# El resultado se almacena en un nuevo DataFrame llamado 'data'
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)

# Eliminar la columna original 'rank' del DataFrame 'data'
data = data.drop('rank', axis=1)

# Estandarizar las características 'gre' y 'gpa' en el DataFrame 'data'
# Calcula la media y la desviación estándar de cada característica y estandariza los valores
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:, field] = (data[field] - mean) / std

# Fijar una semilla para la generación de números aleatorios con fines de reproducibilidad
np.random.seed(42)

# Seleccionar aleatoriamente el 90% de las filas del DataFrame 'data' para el conjunto de entrenamiento
sample = np.random.choice(data.index, size=int(len(data) * 0.9), replace=False)

# Dividir el DataFrame 'data' en dos: 'data' (90%) y 'test_data' (10%)
data, test_data = data.iloc[sample], data.drop(sample)

# Separar las características y los objetivos del conjunto de entrenamiento
features, targets = data.drop('admit', axis=1), data['admit']

# Separar las características y los objetivos del conjunto de prueba
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [3]:
##importamos paquetes y datos
import numpy as np

In [4]:
# Establecer una semilla aleatoria para la reproducibilidad de los resultados.
np.random.seed(21)

In [5]:
# Funciones necesarias
#Calcula la derivada de la función sigmoide en un punto 'x'.
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def sigmoid_prime(x):
    s = sigmoid(x)
    return s*(1-s)
#Calcula la función sigmoide en un punto 'x'.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
# Hyperparámetros
n_hidden = 2  # Número de unidades ocultas en la capa oculta de la red.
epochs = 900  # Número de épocas o iteraciones de entrenamiento.
learnrate = 0.005  # Tasa de aprendizaje, que controla el tamaño de los ajustes de los pesos en cada iteración.

# Obtención del número de entradas (features) y el número de ejemplos (n_records) en los datos de entrada.
n_records, n_features = features.shape
last_loss = None  # Variable para almacenar la pérdida en la última iteración.


# Creamos las matrices de los pesos.
"""
La matriz de pesos que conecta las entradas a las unidades ocultas se inicializa con valores aleatorios que siguen una distribución normal.
La escala de la inicialización se ajusta en función del número de características para evitar gradientes muy grandes o muy pequeños al inicio
del entrenamiento.
"""
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
"""
La matriz de pesos que conecta las unidades ocultas a la capa de salida también se inicializa con valores aleatorios que siguen una distribución normal.
La escala de la inicialización se ajusta en función del número de características.
"""
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

##Entrenamiento

In [8]:
# Iteramos a lo largo de un número específico de épocas (epochs) para entrenar la red neuronal.
for e in range(epochs):
    # Inicializamos los acumuladores de cambios en los pesos para cada época.
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    # Iteramos a través de los datos de entrada y los objetivos en lotes.
    for x, y in zip(features.values, targets):
        ## Forward pass (Paso hacia adelante) ##
        # Calcular la entrada y la salida de la capa oculta y la salida de la red.
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output_layer_in = np.dot(hidden_output, weights_hidden_output)
        output = sigmoid(output_layer_in)

        ## Backward pass (Paso hacia atrás) ##
        # Calcular el error como la diferencia entre la salida real y la predicción de la red.
        error = y - output

        # Calcular el gradiente de error en la unidad de salida.
        output_error = error * sigmoid_prime(output_layer_in)

        # Propagar los errores hacia atrás hasta la capa oculta.
        hidden_error = np.dot(output_error, weights_hidden_output) * sigmoid_prime(hidden_input)

        # Acumular los cambios en los pesos para su posterior ajuste.
        del_w_hidden_output += output_error * hidden_output
        del_w_input_hidden += hidden_error * x[:, None]


    # Actualizar los pesos en cada época.
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.2738503333580284
Train loss:  0.272484852568738
Train loss:  0.2711541129188115
Train loss:  0.26985744940381257
Train loss:  0.2685941900642123
Train loss:  0.2673636578134757
Train loss:  0.2661651721481018
Train loss:  0.26499805074191823
Train loss:  0.26386161092741794
Train loss:  0.2627551710673304
Prediction accuracy: 0.475


##Conclusión
A medida que se ejecuta, se busca minimizar la pérdida en el conjunto de entrenamiento y evaluar la precisión en un conjunto de prueba. Los hiperparámetros y la inicialización de pesos son esenciales en el proceso de entrenamiento y pueden influir en el rendimiento de la red.